In [4]:
!pip install pytube pydub transformers torchaudio librosa moviepy openai-whisper yt_dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.4 MB/s eta 0:00:00


In [5]:
import os
from pytube import YouTube
import whisper
import torch
import yt_dlp

# URL of the YouTube Shorts video
VIDEO_URL = 'https://youtube.com/shorts/4tBQV0tqc3I?si=w0SHBYZKksc8Bl-p'  # Replace with actual URL

# Download the video using yt-dlp
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([VIDEO_URL])

# Load the Whisper model
model = whisper.load_model("base")

# Load the audio file
audio_path = 'audio.mp4'

# Transcribe the audio
result = model.transcribe(audio_path)
transcript = result['text']

print(transcript)


[youtube] Extracting URL: https://youtube.com/shorts/4tBQV0tqc3I?si=w0SHBYZKksc8Bl-p
[youtube] 4tBQV0tqc3I: Downloading webpage
[youtube] 4tBQV0tqc3I: Downloading ios player API JSON
[youtube] 4tBQV0tqc3I: Downloading player 84314bef
[youtube] 4tBQV0tqc3I: Downloading m3u8 information
[info] 4tBQV0tqc3I: Downloading 1 format(s): 247+251
[download] Destination: kadhal kappal vibe girl💃 ｜｜ #instatrending #shorts [4tBQV0tqc3I].f247.webm
[download] 100% of    3.84MiB in 00:00:00 at 10.27MiB/s  
[download] Destination: kadhal kappal vibe girl💃 ｜｜ #instatrending #shorts [4tBQV0tqc3I].f251.webm
[download] 100% of  294.46KiB in 00:00:00 at 2.86MiB/s   
[Merger] Merging formats into "kadhal kappal vibe girl💃 ｜｜ #instatrending #shorts [4tBQV0tqc3I].webm"
Deleting original file kadhal kappal vibe girl💃 ｜｜ #instatrending #shorts [4tBQV0tqc3I].f251.webm (pass -k to keep)
Deleting original file kadhal kappal vibe girl💃 ｜｜ #instatrending #shorts [4tBQV0tqc3I].f247.webm (pass -k to keep)


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 This is the fastest way to live in your life. Wake up, take your phone, see who has texted you. Let others decide how you feel about yourself right after you wake up. F**k water, that's where losers. And whenever you see someone put your head down and start scrolling. Who the f**k would have awkward conversation with strangers? Keep telling yourself that you're an introvert and never, I mean never come out of your comfort zone. Got some work to do? Everyone's got it. So f**k it, let's watch them pawn instead. And where are these people running here? And these people are lifting weights, losers. Yeah, I will replace them. Let's not waste time. Let's play some video games. What some videos. Also, this video seems fun. Let's comment and take it again. F**k, f**k.


In [6]:
from pydub import AudioSegment
import moviepy.editor as mp

# Convert audio to WAV format
audio = AudioSegment.from_file('audio.mp4')
audio.export('audio.wav', format='wav')

# Alternatively, using moviepy to extract audio
# video = mp.VideoFileClip("audio.mp4")
# video.audio.write_audiofile("audio.wav")


<_io.BufferedRandom name='audio.wav'>

In [7]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor
import torch
import librosa

# Load the pre-trained model and feature extractor
model_name = "r-f/wav2vec-english-speech-emotion-recognition"
model = AutoModelForAudioClassification.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Load the audio file
speech_array, sampling_rate = librosa.load('audio.wav', sr=16000)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at r-f/wav2vec-english-speech-emotion-recognition and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
import torch
import torch.nn.functional as F

# Assuming `model` and `feature_extractor` are defined properly in your actual code

# Preprocess the audio
inputs = feature_extractor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

# Move model and inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: inputs[key].to(device) for key in inputs}

# Make predictions
with torch.no_grad():
    logits = model(**inputs).logits

# Process the results
predicted_ids = torch.argmax(logits, dim=-1).item()
emotions = model.config.id2label
scores = F.softmax(logits, dim=1).cpu().numpy()[0]

# Output the results
speech_emotion_output = [{"Emotion": emotions[i], "Score": f"{score * 100:.2f}%"} for i, score in enumerate(scores)]
print(speech_emotion_output)

# Find the most predicted emotion
most_predicted_index = np.argmax(scores)
most_predicted_emotion = emotions[most_predicted_index]
print(f"Most predicted emotion: {most_predicted_emotion}")


[{'Emotion': 'angry', 'Score': '13.38%'}, {'Emotion': 'disgust', 'Score': '13.72%'}, {'Emotion': 'fear', 'Score': '14.21%'}, {'Emotion': 'happy', 'Score': '14.54%'}, {'Emotion': 'neutral', 'Score': '14.76%'}, {'Emotion': 'sad', 'Score': '14.49%'}, {'Emotion': 'surprise', 'Score': '14.91%'}]
Most predicted emotion: surprise


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import whisper  # Assuming this is the library you're using for transcription

# Function to process text and get emotion vector and predicted emotion
def process_text(text, num_labels):
    # Load the emo2vec model and tokenizer for 4 or 6 classes based on num_labels
    if num_labels == 4:
        model_name = "cardiffnlp/twitter-roberta-base-emotion"
        emotion_labels = ['anger', 'joy', 'optimism', 'sad']
    elif num_labels == 6:
        model_name = "bert-base-uncased"  # Use BERT for 6 labels
        emotion_labels = ['anger', 'joy', 'optimism', 'sad', 'fear', 'disgust']
    else:
        raise ValueError("Unexpected number of labels. Please specify either 4 or 6.")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    emotion_probs = F.softmax(logits, dim=-1).squeeze()
    predicted_emotion = emotion_labels[torch.argmax(emotion_probs)]
    emotion_dict = {emotion_labels[i]: emotion_probs[i].item() for i in range(len(emotion_labels))}
    return emotion_dict, predicted_emotion

# Process the transcript for emotions
num_labels = 6  # Change this to 4 or 6 as needed
emotion_dict, predicted_emotion = process_text(transcript, num_labels)

# Print the results
print("Transcript:", transcript)
print("Emotion Probabilities:", emotion_dict)
print("Predicted Emotion:", predicted_emotion)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Transcript:  This is the fastest way to live in your life. Wake up, take your phone, see who has texted you. Let others decide how you feel about yourself right after you wake up. F**k water, that's where losers. And whenever you see someone put your head down and start scrolling. Who the f**k would have awkward conversation with strangers? Keep telling yourself that you're an introvert and never, I mean never come out of your comfort zone. Got some work to do? Everyone's got it. So f**k it, let's watch them pawn instead. And where are these people running here? And these people are lifting weights, losers. Yeah, I will replace them. Let's not waste time. Let's play some video games. What some videos. Also, this video seems fun. Let's comment and take it again. F**k, f**k.
Emotion Probabilities: {'anger': 0.17448261380195618, 'joy': 0.17655262351036072, 'optimism': 0.14043685793876648, 'sad': 0.2100958675146103, 'fear': 0.15233784914016724, 'disgust': 0.1460941731929779}
Predicted Emot

In [10]:
import numpy as np

# # Speech emotion recognition output
# speech_emotion_output = scores

# # Text emotion recognition output
# text_emotion_output = emotion_dict

# Combine the emotion probabilities from speech and text models
speech_emotion_probs = np.array([float(d['Score'].replace('%', '')) / 100 for d in speech_emotion_output])
text_emotion_probs = np.array(list(emotion_dict.values()))

# Ensure the arrays have the same length
max_length = max(len(speech_emotion_probs), len(text_emotion_probs))
speech_emotion_probs = np.pad(speech_emotion_probs, (0, max_length - len(speech_emotion_probs)), mode='constant', constant_values=0)
text_emotion_probs = np.pad(text_emotion_probs, (0, max_length - len(text_emotion_probs)), mode='constant', constant_values=0)

# Combine the emotion probabilities
combined_emotion_probs = (speech_emotion_probs + text_emotion_probs) / 2

# Determine the most predicted emotion
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise', 'anger', 'joy', 'optimism', 'sadness', 'fear', 'disgust']
most_predicted_index = np.argmax(combined_emotion_probs)
most_predicted_emotion = emotion_labels[most_predicted_index]

# Create the combined output emotion vector with labels
combined_emotion_vector = [f"{emotion_labels[i]}:{combined_emotion_probs[i]:.4f}" for i in range(len(combined_emotion_probs))]


print("Combined Emotion Vector:", combined_emotion_vector)
print("Most predicted emotion:", most_predicted_emotion)


Combined Emotion Vector: ['angry:0.1541', 'disgust:0.1569', 'fear:0.1413', 'happy:0.1777', 'neutral:0.1500', 'sad:0.1455', 'surprise:0.0746']
Most predicted emotion: happy
